In [136]:
from datasets import load_dataset
from gensim.utils import tokenize
from tqdm import tqdm
from textblob import TextBlob
from collections import Counter
from torch.optim import AdamW
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
data=load_dataset('imdb')
device='cuda'
from torch.nn import Module
from torch import nn 

In [77]:
all_sentences=[]
sent_threshold=32

for text_block in tqdm(data['train']['text']):
    for sentences in TextBlob(text_block).sentences:
        len_of_sent=sentences.words 
        for sent in sentences.split('.<br /><br />'):
            if len(len_of_sent)<sent_threshold:
                all_sentences.append(sent)
    
    


100%|███████████████████████████████████████████████████████████| 25000/25000 [00:22<00:00, 1095.55it/s]


In [137]:
words=[]
for sent in tqdm(all_sentences):
    for word in tokenize(sent):
        words.append(word.lower())

100%|███████████████████████████████████████████████████████| 227434/227434 [00:01<00:00, 140798.09it/s]


In [138]:
cntr=Counter(words)

In [139]:
vocab_size=40000
our_words=set(['<unk>','<bos>','<eos>','<pad>'])
for word,cnt in cntr.most_common()[0:vocab_size]:
    our_words.add(word)

In [140]:
len(our_words)

40004

In [141]:
for i in tokenize(all_sentences[5]):
    print(i)

I
Am
Curious
Yellow
is
a
risible
and
pretentious
steaming
pile


In [142]:
word2int={word:cnt for cnt,word in enumerate(our_words)}
int2word={cnt:word for cnt,word in enumerate(our_words)}

In [143]:
class word_dataset(Dataset):
    def __init__(self,data):
        super(word_dataset,self).__init__()
        self.data=data

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        selected_sent=self.data[idx]
        indexed_text=[word2int['<bos>']]
        main_text=[word2int[word] if word in word2int else word2int['<unk>'] for word in tokenize(selected_sent.lower())]
        indexed_text=indexed_text+main_text+[word2int['<eos>']]
        return indexed_text
        
        
        

In [144]:
def make_batch(batch):
    lenghts=(len(x) for x in batch)
    max_len=max(lenghts)

    new_batch=[]
    for sent in batch:
        for pad in range(max_len-len(sent)):
            sent.append(word2int['<pad>'])
        new_batch.append(sent)

    new_batch=torch.LongTensor(new_batch).to(device)

    

    return {
        'inp':new_batch[:,:-1],
        'label':new_batch[:,1:]
    }
    

In [145]:
train_data=word_dataset(all_sentences)
train_dataloader=DataLoader(train_data,batch_size=32,shuffle=True,collate_fn=make_batch)

In [146]:
class LanguageModel(Module):
    def __init__(self,vocab_size,hidden_size):
        super().__init__()

        self.emb=nn.Embedding(vocab_size,hidden_size)
        self.gru=nn.GRU(hidden_size,hidden_size,num_layers=3,batch_first=True)

        self.lay_with_drop=nn.Sequential(
            nn.Tanh(),
            nn.Linear(hidden_size,hidden_size),
            nn.Dropout(),
            nn.Tanh()
        )

        self.final_lin=nn.Sequential(
            nn.Linear(hidden_size,vocab_size)
        )

        
    def forward(self,text):
        
        emb_x=self.emb(text)
        
    
        x,_=self.gru(emb_x)
        

        #agr_x=x.mean(dim=1)#???????
        

        x=self.lay_with_drop(x)
        x=self.final_lin(x)
        return x
        

        

In [147]:
model=LanguageModel(len(our_words),128).to(device)
loss_fn=nn.CrossEntropyLoss(ignore_index=word2int['<pad>'])
optimizer=AdamW(model.parameters())

In [148]:
len(our_words)

40004

In [149]:
def train(model,loss_fn,optimizer,dataloader):
    for epoch in epochs:
        for batch in (pbar:=tqdm(dataloader)):
            optimizer.zero_grad()
            pred=model(batch['inp']).flatten(start_dim=0,end_dim=1)
            
            loss=loss_fn(pred,batch['label'].flatten())
            loss_item=loss.item()
            loss.backward()
            optimizer.step()
            pbar.set_description(f'loss:{loss_item}')

In [150]:
train(epochs,model,loss_fn,optimizer,train_dataloader)

NameError: name 'epochs' is not defined

In [183]:
def check_the_acc(model,starting_seq,max_seq_len=128):
    model=model.to(device)
    input_ids=[word2int['<bos>']] +[word2int[word] if word in word2int else word2int['<unk>'] for word in starting_seq.split()]
    print(input_ids)
    input_ids=torch.LongTensor(input_ids).to(device)
    print(input_ids)#добавил унсквииз
    with torch.no_grad():
        for i in range(max_seq_len):
            print(input_ids[-1])
            new_word_dist=model(input_ids[-1].unsqueeze(dim=0))
            new_word=new_word_dist.squeeze().argmax()
            input_ids=torch.cat([input_ids,new_word.unsqueeze(0)])
            print(1)

In [184]:
check_the_acc(model,'Привет мир',5)

[4839, 22334, 22334]
tensor([ 4839, 22334, 22334], device='cuda:0')
tensor(22334, device='cuda:0')
1
tensor(20003, device='cuda:0')
1
tensor(1940, device='cuda:0')
1
tensor(10343, device='cuda:0')
1
tensor(28377, device='cuda:0')
1
